In [24]:
import networkx as nx
import acopy
import time

## Question 2

In [3]:
# Question 2b

g_berlin = nx.read_gml('../../data/tsp/../../data/tsp/berlin52.gml')
g_brazil = nx.read_gml('../../data/tsp/../../data/tsp/brazil58.gml')
g_brg = nx.read_gml('../../data/tsp/../../data/tsp/brg180.gml')
g_gr = nx.read_gml('../../data/tsp/../../data/tsp/gr229.gml')
g_pr = nx.read_gml('../../data/tsp/../../data/tsp/pr439.gml')

print('g_berlin has', g_berlin.number_of_nodes(), 'nodes and', g_berlin.number_of_edges(), 'edges')
print('g_brazil has', g_brazil.number_of_nodes(), 'nodes and', g_brazil.number_of_edges(), 'edges')
print('g_brg has', g_brg.number_of_nodes(), 'nodes and', g_brg.number_of_edges(), 'edges')
print('g_gr has', g_gr.number_of_nodes(), 'nodes and', g_gr.number_of_edges(), 'edges')
print('g_pr has', g_pr.number_of_nodes(), 'nodes and', g_pr.number_of_edges(), 'edges')

g_berlin has 52 nodes and 2652 edges
g_brazil has 58 nodes and 3306 edges
g_brg has 180 nodes and 32220 edges
g_gr has 229 nodes and 52212 edges
g_pr has 439 nodes and 192282 edges


In [4]:
# alpha affects pheromone importance
# beta affects distance/weight importance
# rho is pheromone evaporation
# Q is amount of pheromone ants release 
solver = acopy.Solver(rho=.03, q=1)
colony = acopy.Colony(alpha=1, beta=5) 

It seems like the package needs distances to be coded as 'weight' so I need to rename the 'cost' attribute to 'weight'

In [23]:
def rename_cost_weight(G):
    # Iterate through all edges in G
    for u, v, attrs in G.edges(data=True):
        if 'cost' in attrs:
            # Rename 'cost' to 'weight'
            attrs['weight'] = attrs.pop('cost')

# # Check the first 5 edges to ensure 'cost' has been renamed to 'weight'
rename_cost_weight(g_berlin)
print(list(g_berlin.edges(data=True))[:5])
rename_cost_weight(g_brazil)
rename_cost_weight(g_brg)
rename_cost_weight(g_gr)
rename_cost_weight(g_pr)


[('0', '1', {'pheromone': 0.00022344521551333316, 'weight': 666.1080993352356}), ('0', '2', {'pheromone': 0.0021771484545130633, 'weight': 281.1138559374119}), ('0', '3', {'pheromone': 2.0788686828312823e-05, 'weight': 395.6008088970497}), ('0', '4', {'pheromone': 3.622976025985899e-05, 'weight': 291.2043955712207}), ('0', '5', {'pheromone': 7.460483618897342e-05, 'weight': 326.266762021509})]


In [16]:
tour = solver.solve(g_berlin, colony, limit=100)

In [17]:
tour.cost

7570.497072025889

In [18]:
rhos = linspace(0,0.1,0.02)
for i in rhos:
    

7570.497072025889	0,21,30,17,2,16,20,41,1,6,29,22,19,49,28,15,45,43,33,34,35,38,39,37,36,47,23,4,14,5,3,24,11,27,26,25,46,12,13,51,10,50,32,42,9,8,7,40,18,44,31,48